In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity_recognition_exp.zip' -d '/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /'

In [ ]:
data1 = pd.read_csv('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/Phones_accelerometer.csv')		#Loading Accelerometer data
data2 = pd.read_csv('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/Phones_accelerometer.csv')			#Loading Gyroscope data

In [ ]:
length1 = len(data1)
length2 = len(data2)
length = min(length1, length2)						#To make the length of the merged data equal to minimum of the two data 


In [ ]:
data1 = data1.drop(labels = ['Arrival_Time','Creation_Time','Index', 'User'], axis=1)		#Dropping the unnecessary fields
data2 = data2.drop(labels = ['Arrival_Time','Creation_Time','Index', 'User','Model','Device'], axis=1)


In [ ]:
data2.columns = ['x1', 'y1', 'z1', 'gt1']

In [ ]:
data = pd.concat([data1, data2], axis=1)

In [ ]:
to_drop = ['null']									#To drop the null values fro both data1 and data2
data = data[~data['gt'].isin(to_drop)]
data = data[~data['gt1'].isin(to_drop)]

In [ ]:
data = data.drop(labels = ['gt1'], axis=1)

In [ ]:
cols_to_norm = ['x','y','z', 'x1','y1','z1']		#Normalizing the columns
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [ ]:
y = data[['gt']]									#Extracting only the action values
y= pd.get_dummies(y)


In [ ]:
sample_rate = 50  # number of observation per second based on dataset documentation

sliding_size = int(.2 * sample_rate)  # number of skipped datapoints to start next window
win_size=128

In [ ]:
def sliding_window(overlap=False):
 
  if overlap:
        step_size = sliding_size  # for Overlapping technique
  else:

    step_size = win_size  # for Non-overlapping technique
  y_list=[]
  x_list=[]      
  for i in range(0, data.shape[0], step_size):
    start = i
    end = start + step_size
    if end<data.shape[0]:
      
      label = data.loc[i]['gt']
      X_point = np.zeros((step_size,6))
     
      X_point[:,0] = data.iloc[i: i + step_size, 0]
      X_point[:,1] = data.iloc[i: i + step_size, 1]
      X_point[:,2] = data.iloc[i: i + step_size, 2]
      X_point[:,3] = data.iloc[i: i + step_size, 6]
      X_point[:,4] = data.iloc[i: i + step_size, 7]
      X_point[:,5] = data.iloc[i: i + step_size, 8]
      x_list.append(X_point)
      y_list.append(label)
   
  return y_list,  x_list 


         

In [ ]:
overlap=True
step_size=None
if overlap:
        step_size = sliding_size  # for Overlapping technique
else:
  step_size = win_size  # for Non-overlapping technique

In [ ]:
Y,X=sliding_window(overlap)

In [ ]:
X_train = np.zeros((len(Y),step_size,6))
X_train[:,:]=[x for x in X]

In [ ]:
y_df=df = pd.DataFrame(Y)

In [ ]:
y_dummy= pd.get_dummies(y_df)

In [ ]:
y_np = np.array(y_dummy)

In [ ]:
Y_train=np.zeros(len(Y))
for y in y_np:
   index=np.where(y == 1)
   print(index)
   Y_train[:]=index

In [ ]:
np.savez_compressed('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /UCI_HHAR_X.npz', X_train)

In [ ]:
np.savez_compressed('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /UCI_HHAR_Y.npz', y_np)